In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

# Prepare the text and create character to integer mapping
raw_text = "000100010001000100010001000100010001000100010001000100010001000100010001"
raw_text = raw_text.lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

# Convert characters to integers
int_text = [char_to_int[char] for char in raw_text]

# Prepare input and target sequences for training
input_seq = torch.tensor(int_text, dtype=torch.float32).reshape(-1, 1, 1)
target_seq = torch.tensor(int_text, dtype=torch.long).reshape(-1, 1, 1)

# Create a list of tuples (input, target) for training
train_data = [(input_seq[i], target_seq[i]) for i in range(len(input_seq))]


class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.out = ''

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out
    def test_model(self, input_str, char_to_int, int_to_char):
      model.eval()  # Set the model to evaluation mode
      input_str = input_str.lower()  # Convert string to lowercase
      int_text = [char_to_int[char] for char in input_str]  # Convert input string to integer sequence

      output_str = ""  # Initialize the output string

      with torch.no_grad():  # Deactivate gradients for the following code
          for i in int_text:
              # Prepare the input tensor and reshape it
              input_tensor = torch.tensor(i, dtype=torch.float32).reshape(1, 1, 1)
              # Forward pass
              output = model(input_tensor)
              # Get the index of the max value among the predicted character probabilities
              _, predicted_idx = torch.max(output, 1)
              # Convert the predicted index to its corresponding character and add it to the output string
              output_str += int_to_char[predicted_idx.item()]

      return output_str


# Initialize the model, loss, and optimizer
# Hidden layer Hyperparameter defined here
input_dim = 1
hidden_dim = 160
output_dim = len(chars)  # Number of unique characters

model = SimpleLSTM(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

# Train the model
# Set number of epochs here
for epoch in range(10):
    for seq, label in train_data:
        seq = seq.view(1, 1, 1)
        label = label.view(1)

        optimizer.zero_grad()
        output = model(seq)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


# Create an integer to character mapping
int_to_char = dict((i, c) for i, c in enumerate(chars))

# Test the model with a new string
result = model.test_model("0100", char_to_int, int_to_char)
print(f"The predicted next characters are: {result}")






Epoch 1, Loss: 0.0005199749139137566
Epoch 2, Loss: 0.0002455409849062562
Epoch 3, Loss: 0.00015889335190877318
Epoch 4, Loss: 0.00011264643399044871
Epoch 5, Loss: 8.40390202938579e-05
Epoch 6, Loss: 6.48477507638745e-05
Epoch 7, Loss: 5.1377883210079744e-05
Epoch 8, Loss: 4.172238186583854e-05
Epoch 9, Loss: 3.4450891689630225e-05
Epoch 10, Loss: 2.884823152271565e-05
The predicted next characters are: 0100
